In [13]:
import numpy as np
import pandas as pd
import re

In [14]:
df = pd.read_csv(r'../datasets/reddit_dataframe.csv')

Removing nulls which somehow get into the dataframe:

In [15]:
df[['subreddit','selftext']].isna().sum()

subreddit    0
selftext     1
dtype: int64

In [16]:
null_index = list(df[df[['selftext', 'subreddit']].isnull().any(axis = 1)].index)
df.drop(index = null_index, inplace = True)
df.index = range(len(df))

In [17]:
df[['subreddit','selftext']].isna().sum()

subreddit    0
selftext     0
dtype: int64

In [18]:
test = df['selftext'].loc[5032]
test

'I currently work a outdoor job, nothing crazy and the pay isn’t great, but with how terrible the market has been for me, I can’t seem to get back into a field I’m skilled in. Just for context, most of my work history is related to pc hardware and software diagnostics, cellular repair and performing IT for people. I also done some data entry for a business. I took a break from that field of work over six months ago and decided to try a different profession. I got into pest control, so for the next six months I worked with a pest control company that had extremely toxic management, which included verbal harassment/belittling and very poor training for the job. The pay I was given for this job was 17.00. So after dealing with this, I decided I wanted to go back into tech industry, I have years of experience in this stuff and I thought it wouldn’t be too difficult to do. A local franchise that specializes in tech repair wanted me to come in for two interviews. Had both done and was inform

It's very common to see posts in the dataset which have newline or tab characters. I'm going to do some basic cleaning using regex to get rid of any glaring issues. I can do further cleaning during the EDA phase.

https://stackoverflow.com/questions/16355732/how-to-remove-tabs-and-newlines-with-a-regex

In [19]:
re.sub(r"\s+", " ", test)

'I currently work a outdoor job, nothing crazy and the pay isn’t great, but with how terrible the market has been for me, I can’t seem to get back into a field I’m skilled in. Just for context, most of my work history is related to pc hardware and software diagnostics, cellular repair and performing IT for people. I also done some data entry for a business. I took a break from that field of work over six months ago and decided to try a different profession. I got into pest control, so for the next six months I worked with a pest control company that had extremely toxic management, which included verbal harassment/belittling and very poor training for the job. The pay I was given for this job was 17.00. So after dealing with this, I decided I wanted to go back into tech industry, I have years of experience in this stuff and I thought it wouldn’t be too difficult to do. A local franchise that specializes in tech repair wanted me to come in for two interviews. Had both done and was inform

In [20]:
# implementing newline character removal
df['selftext'] = [re.sub(r"\s+", " ", s) for s in df['selftext']]

In [21]:
[s for s in df['selftext'] if "A place I applied to i" in s][0]

"Hi A place I applied to in the past recently reached out and sent me a job offer for a new position they have opening up. It feels nice to be wanted, lol, and I know I would be good at it, but I just wasn't expecting it and I've been pretty comfortable at my current job. I make $23/hr now and they offered $26. The job description isn't that different from my current job, but the place I work at is so weird and unprofessional that I've gotten used to it and the idea of going to an office that's more normal and professional doesn't sound very appealing? My current job is a family business where I can wear whatever I want and basically do whatever I want as long as I'm getting my work done. When it's slow I get to draw and work on my hobbies at my desk as long as it's not excessive. The actual workplace is technically super dysfunctional and sometimes feels unstable but my position gets by pretty unscathed. So idk. The increase in pay at the new job would basically cancel out my student 

In [24]:
df['selftext'].iloc[94]

'My boyfriend worked at a national furniture chain store for a out 6 months. He quit after they continually pressured him from hourly + commission to strictly commission. He put in his two weeks notice on June 27. He worked through the July 1 and was scheduled next on July 4 but had to call off. He was then asked if he wanted to be on the next week\'s schedule (which started the next day but wasn\'t done yet), he said he didn\'t care. They didn\'t put him on the schedule. Fast forward to this week. He goes to pick up his June commissions check and they don\'t have it. Apparently the manager who told him he was not scheduling him instead labelled him as abandoning the job or something, which gives them cause to not pay his commission. Apparently that manager quit later that same week. Other managers were "sympathetic" but couldn\'t help. I say that\'s wage theft. We contacted the state online. Does he have a chance of seeing that money? What other steps can/should we take? His sales las

I happened upon this while looking for tab symbols. I should filter this out at as well.

Filtering out banned posts: 
NOTE: This only outputed on the first time ran. Since the post is now gone, it doesn't show up anymore. It was at index 94 and there was only 1 instance.

In [ ]:
[s for s in df['selftext'] if 'Removed by Reddit' in s][0]

There's only one item with 'removed by Reddit" in the selftext:

In [46]:
# originally output 1
len([s for s in df['selftext'] if 'Removed by Reddit' in s])

0

I'll remove it manually.

In [27]:
df.shape

(6620, 116)

In [28]:
df.drop(index = 94, inplace = True)
df.index = range(len(df))

In [29]:
df.shape

(6619, 116)

In [30]:
len([s for s in df['selftext'] if 'Removed by Reddit' in s])

0

Another issue is that there are some artifacts like `&amp;#x200B;` which are the result of certain characters not being handled correctly.

In [31]:
[s for s in df['selftext'] if '&amp;' in s][10]

'I watched Norma Rae for the first time. It is on Max if you have a subscription. I also found it streaming on [Vimeo](https://vimeo.com/522567048). The film was released in 1979 (right before the devil took office in 1981) and is about a cotton mill worker and single mom who decides that she has no choice, but to fight for a union. It stars Sally Fields and Beau Bridges. It won best actress and best picture at the Academy Awards that year. Norma Rae is based on Crystal Lee Sutton. I do not know much about her. (No surprise that schools do not teach about labor organizers in school). Wikipedia states that the fight between union organizers and the company was one of "the ugliest episodes in labor history ... from 1963-1980". Sutton and other labor organizers was repeatedly harassed and fired. "In 1973 Crystal saw a union poster hanging in one of the seven mills in Roanoke Rapids, North Carolina owned by J.P. Stevens &amp; Company mills where three generations of her family had worked—l

`&amp;` is meant to be `&`:

In [32]:
amp_index = [s for s in range(len(df['selftext'])) if '&amp;' in df['selftext'][s]]

In [33]:
df['selftext'] = [s.replace('&amp;', '&') for s in df['selftext']]
len([s for s in df['selftext'] if '&amp;' in s])

0

`&amp;#x200B;` characters have now been turned into `&#x200B;` characters.

In [34]:
[s for s in df['selftext'] if '&#x200B;' in s][8]

"Hi, I interviewed for a role of a place I was enthusiastic for. I sent in my CV, and got a very quick response for an interview. I learned that the interview was in stages, and it seems like I was a pretty good match for the role. In most of my previous jobs, I had a one or two-stage interview (occasionally a third to negotiate the final details). So I go on the interview, it was online, it took like an hour, I think it was good. After I was asked to meet the team online, so I joined in for that meeting, but it was also like an interview. So two good interviews, one with a manager and one with the team - I thought I was in the clear. Wrong. X. Nope. not yet. I had to have an interview with another director, which was also a long interview. I'm starting to feel like I am personally evaluated, but ok, it's good to meet the whole team. I have worked in a lot of awesome jobs before so I thought this was in the bag. Heck I had three long interviews with everyone. Nope, not yet, I get a req

I'll use Regex to remove any string starting with & and ending with ;

In [35]:
df['selftext'] = [re.sub('\s*&#x200B;\s*', '', s) for s in df['selftext']]

In [36]:
[s for s in df['selftext'] if ';' in s][8]

'Y\'all are just lazy, get some backbone... &lt;ahem&gt; JUST KIDDING! Seriously, I\'m at the "end stage" of my work career, most of which has been as an IT or infrastructure contractor but I\'ve also done a LOT of other random work during those occasional periods when the good contracts were scarce and here is what I\'ve learned from that. 1) Jobs come and go. Sometimes they go because of something YOU did. In that case "learn and do better" and then move on. Mostly jobs go away because of something completely out of your control regardless of the blame assigned by whomever is doing the firing. When that is the case, walk away and take some time to recover and then move on. 2) A "support network" is an absolute necessity. Maybe it\'s family, maybe friends, maybe a church, or a mix of all that and more, but no matter what happens or how bad you feel don\'t cut off your support network because you\'ll need it and because the most surprising opportunities can arise from the most random i

In [37]:
[re.sub(r"\'", "'", s) for s in df['selftext']][3]

'My company leadership said they wanted to buy me lunch with some of the staff to "Thank me for my years of work." I\'m being outsourced in two weeks and didn\'t get offered other positions or anything. In a nutshell, my employment is being terminated. I said thanks, but I\'m not interested in getting lunch to celebrate being fired.'

In [38]:
[s for s in df['selftext'] if "can\'t" in s][8]

'The video: [https://www.youtube.com/watch?v=YVTxute2lhA](https://www.youtube.com/watch?v=YVTxute2lhA) I wrote out a quite lengthy response to all of their points. 0:00 "The US economy still has a massive massive labor shortage." • No it doesn\'t. It has a wage shortage. Companies have plenty of money to pay people more, but they refuse. No one wants to put up with horrible jobs that pay nothing when rent is sky high.0:09 "obviously the pandemic exacerbated the problem." • The cost of living and company profits going up year after year for decades, but average wages hardly going up at all is the main problem. Wrong again.0:52 "600 thousand nurses plan to leave the field by 2027." • 7,000 nurses walked out in New York because their company failed to reach a deal. Hospitals charge people $100,000 for an aspirin and a band-aid, yet refuse to pay the people who run the show. It\'s not a labor shortage - it\'s a wage shortage.1:04 "4% of all public school teaching positions are empty." • It

In [39]:
def replace_escaped_single_quotes(text):
    return re.sub(r"\\'", "'", text)

df['selftext'] = df['selftext'].apply(replace_escaped_single_quotes)

In [40]:
[s for s in df['selftext'] if ';' in s][8]

'Y\'all are just lazy, get some backbone... &lt;ahem&gt; JUST KIDDING! Seriously, I\'m at the "end stage" of my work career, most of which has been as an IT or infrastructure contractor but I\'ve also done a LOT of other random work during those occasional periods when the good contracts were scarce and here is what I\'ve learned from that. 1) Jobs come and go. Sometimes they go because of something YOU did. In that case "learn and do better" and then move on. Mostly jobs go away because of something completely out of your control regardless of the blame assigned by whomever is doing the firing. When that is the case, walk away and take some time to recover and then move on. 2) A "support network" is an absolute necessity. Maybe it\'s family, maybe friends, maybe a church, or a mix of all that and more, but no matter what happens or how bad you feel don\'t cut off your support network because you\'ll need it and because the most surprising opportunities can arise from the most random i

In [41]:
def replace_escaped_characters(text):
    return re.sub(r"\\(['])", r"\1", text)

df['selftext'] = df['selftext'].apply(replace_escaped_characters)
df['selftext'][3]

'My company leadership said they wanted to buy me lunch with some of the staff to "Thank me for my years of work." I\'m being outsourced in two weeks and didn\'t get offered other positions or anything. In a nutshell, my employment is being terminated. I said thanks, but I\'m not interested in getting lunch to celebrate being fired.'

In [42]:
test = df['selftext'][3]
test.replace("\'", "'")

'My company leadership said they wanted to buy me lunch with some of the staff to "Thank me for my years of work." I\'m being outsourced in two weeks and didn\'t get offered other positions or anything. In a nutshell, my employment is being terminated. I said thanks, but I\'m not interested in getting lunch to celebrate being fired.'

This isn't working as expected. Using raw strings or double backslashes doesn't work either, although interestingly, a double quotation mark does work. Although this isn't what I want.

In [43]:
[s.replace("\'", '"') for s in df['selftext']][3]

'My company leadership said they wanted to buy me lunch with some of the staff to "Thank me for my years of work." I"m being outsourced in two weeks and didn"t get offered other positions or anything. In a nutshell, my employment is being terminated. I said thanks, but I"m not interested in getting lunch to celebrate being fired.'

In [44]:
df.to_csv('../datasets/reddit_dataframe.csv', index_label = False)